In [ ]:
#@title Importing libraries (double-click to show code)
import os
import pkg_resources
pkg_resources.require("numpy>=`1.20.1")  #  to use specific numpy version
import numpy as np
import rsnapsim as rss
import sys
from sys import platform
from skimage import io ; from skimage.io import imread; from skimage.measure import find_contours

from random import randrange
import pandas as pd
import os; from os import listdir; from os.path import isfile, join
import re # to iterate in files
import glob # to iterate in files
import shutil
import pathlib
from random import randrange
import ipywidgets as widgets ; from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox, Layout, GridspecLayout 
from IPython.display import Image
import dna_features_viewer
from dna_features_viewer import BiopythonTranslator, GraphicFeature, GraphicRecord, CircularGraphicRecord
import tifffile

# Deffining directories
current_dir = pathlib.Path().absolute()
sequences_dir = current_dir.parents[1].joinpath('DataBases','gene_files')
video_dir = current_dir.parents[1].joinpath('DataBases', 'LargeDataSets' ,'EmptyFISH')
#video_dir = current_dir.parents[1].joinpath('DataBases', 'LargeDataSets' ,'EmptyFISH')
rsnaped_dir = current_dir.parents[1].joinpath('rsnaped')

# Importing rSNAPsim_IP
sys.path.append(str(rsnaped_dir))
import rsnaped as rsp
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('default')
plt.rcParams['figure.dpi'] = 120
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['axes.grid'] = False

In [ ]:
list_files_names = sorted([f for f in listdir(video_dir) if isfile(join(video_dir, f)) and ('.tif') in f], key=str.lower)  # reading all tif files in the folder
list_files_names.sort(key=lambda f: int(re.sub('\D', '', f)))  # sorting the index in numerical order
path_files = [ str(video_dir.joinpath(f).resolve()) for f in list_files_names ] # creating the complete path for each file
num_files = len(path_files)

In [ ]:
path_files

In [ ]:
list_files_names

In [ ]:
# This function takes all the videos in the folder and merge those with similar names
list_prefix = [] # list that will store name prefix for every image
ending_string = re.compile('.*_C0.tif')  # detecting files ending in _C0.tif
for root, dirs, files in os.walk(video_dir):
  for file in files:
    if ending_string.match(file):
        prefix = file.rpartition('_')[0]
        print(file)
        list_prefix.append(prefix)

In [ ]:
list_prefix

In [ ]:
# This function takes all the videos in the folder and merge those with similar names
list_files_per_image =[]  # list that stores all files belonging to the same image in a sublist
ending_string = re.compile('.*_C0.tif')  # detecting files ending in _C0.tif
for _, _, files in os.walk(video_dir):
  for file in files:
    if ending_string.match(file):
        prefix = file.rpartition('_')[0]  # stores a string with the first part of the file name before the last underscore character in the file name string.
        list_files_per_image.append(glob.glob( str(video_dir.joinpath(prefix)) + '*.tif'))

In [ ]:
list_files_per_image[1]

In [ ]:
number_images = len(list_files_per_image)


In [ ]:
#list_images_separated_ch = [imread(f) for f in files]

In [ ]:
# merging files for single image
test_idx= 1
merged_image = np.concatenate([ imread(list_files_per_image[test_idx][i])[..., np.newaxis] for i,_ in enumerate(list_files_per_image[test_idx])],axis=-1)
merged_image.shape

In [ ]:
# Storing a list with all merged images

In [ ]:
# This function takes all the videos in the folder and merge those with similar names
list_file_names =[]
list_merged_images =[]  # list that stores all files belonging to the same image in a sublist
ending_string = re.compile('.*_C0.tif')  # detecting files ending in _C0.tif
for _, _, files in os.walk(video_dir):
  for file in files:
    if ending_string.match(file):
        prefix = file.rpartition('_')[0]  # stores a string with the first part of the file name before the last underscore character in the file name string.
        list_files_per_image = glob.glob( str(video_dir.joinpath(prefix)) + '*.tif')
        #merged_image = np.concatenate([ imread(list_files_per_image[i])[..., np.newaxis] for i,_ in enumerate(list_files_per_image)],axis=-1)
        list_file_names.append(prefix)
        list_merged_images.append(np.concatenate([ imread(list_files_per_image[i])[..., np.newaxis] for i,_ in enumerate(list_files_per_image)],axis=-1))


In [ ]:
def merging_files(directory, substring_to_detect_in_file_name = '.*_C0.tif', save_figure =0 ):
    # This function takes all the videos in the folder and merge those with similar names
    list_file_names =[]
    list_merged_images =[]  # list that stores all files belonging to the same image in a sublist
    ending_string = re.compile(substring_to_detect_in_file_name)  # detecting files ending in _C0.tif
    for _, _, files in os.walk(directory):
        for file in files:
            if ending_string.match(file):
                prefix = file.rpartition('_')[0]  # stores a string with the first part of the file name before the last underscore character in the file name string.
                list_files_per_image = glob.glob( str(directory.joinpath(prefix)) + '*.tif')
                list_file_names.append(prefix)
                merged_img = np.concatenate([ imread(list_files_per_image[i])[..., np.newaxis] for i,_ in enumerate(list_files_per_image)],axis=-1).astype('uint16')
                print(merged_img.shape)
                list_merged_images.append(merged_img) 
                if save_figure ==1:
                    save_to_path = directory.joinpath('merged')
                    if not os.path.exists(str(save_to_path)):
                        os.makedirs(str(save_to_path))
                    tifffile.imsave(str(save_to_path.joinpath(prefix+'_merged'+'.tif')), merged_img, metadata={'axes': 'ZYXC'})
    return list_file_names, list_merged_images

In [ ]:
list_file_names, list_merged_images = merging_files(video_dir, substring_to_detect_in_file_name = '.*_C0.tif', save_figure =0)

In [ ]:
#plt.imshow(list_merged_images[0][0,:,:,0])
list_merged_images[0].shape

In [ ]:
#imported_img_merged = imread(str(video_dir.joinpath('merged','ROI002_XY1620757341_Z00_T0_merged.tif')))
#plt.imshow(imported_img_merged[0,:,:,0])
#imported_img_merged.shape

In [ ]:
 #tifffile.imsave('test.tif', list_merged_images[0], metadata={'axes': 'ZYXC'})

In [ ]:
#str(video_dir.joinpath('merged','ROI002_XY1620757341_Z00_T0_merged.tif'))

In [ ]:
#video_dir.joinpath('merged','ROI002_XY1620757341_Z00_T0_merged.tif')

In [ ]:
list_file_names, list_merged_images = rsp.MergeChannels(video_dir, substring_to_detect_in_file_name = '.*_C0.tif', save_figure =1).merge()

In [ ]:
list_file_names

In [ ]:
list_merged_images[0].shape